## 0.0 IMPORTS

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from scipy.stats import kurtosis, skew, norm
import math
import statistics
from scipy.stats import chi2

pd.set_option('float_format', '{:.3f}'.format)

## 0.1 - HELPER FUNCTION (FUNÇÕES AUXILIARES)

In [2]:
def show_descriptive_statistical(df):
    # Central Tendency - mean, median
    ct1 = pd.DataFrame(df.apply(np.mean)).T
    ct2 = pd.DataFrame(df.apply(np.median)).T

    # Dispersion - std, min, max, range, skew, kurtosis
    d1 = pd.DataFrame(df.apply(np.std)).T
    d2 = pd.DataFrame(df.apply(min)).T
    d3 = pd.DataFrame(df.apply(max)).T
    d4 = pd.DataFrame(df.apply(lambda x: x.max() - x.min())).T
    d5 = pd.DataFrame(df.apply(lambda x: x.skew())).T
    d6 = pd.DataFrame(df.apply(lambda x: x.kurtosis())).T

    m = pd.concat([d2, d3, d4, ct1, ct2, d1, d5, d6]).T.reset_index()
    m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
    return m

## 0.2 - PATH DEFINITION

In [4]:
HOME_PATH = '/home/valmir/Documentos/deteccao_fraude/'
RAW_DATA_PATH = 'data/raw/'
INTERIM_DATA_PATH = 'data/intermediario/'
FIGURE_PATH = 'reports/figures/'

## 0.3 - LOAD DATA

In [5]:
base_fraude_raw = pd.read_csv(HOME_PATH+RAW_DATA_PATH+'fraud_detection_dataset.csv')
base_fraude_raw.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9920.520,C764826684,0.000,0.000,M1940055334,0.000,0.000,0,0
1,1,PAYMENT,3448.920,C2103763750,0.000,0.000,M335107734,0.000,0.000,0,0
2,1,PAYMENT,4206.840,C215078753,0.000,0.000,M1757317128,0.000,0.000,0,0
3,1,PAYMENT,5885.560,C840514538,0.000,0.000,M1804441305,0.000,0.000,0,0
4,1,PAYMENT,5307.880,C1768242710,0.000,0.000,M1971783162,0.000,0.000,0,0


## 1.0 - DATA DESCRIPITION

In [8]:
#base_fraude_raw1 = base_fraude_raw.copy()
#base_fraude_raw1.to_csv(HOME_PATH+INTERIM_DATA_PATH+'base_fraude_raw1.csv')
base_fraude_raw1 = pd.read_csv(HOME_PATH+INTERIM_DATA_PATH+'base_fraude_raw1.csv')


## 1.1 - DATA FIELD'S

Os dados contidos nesta base de dados são simulados e compreendem 30 dias. Estas transações podem ser de diferentes tipo e foram executadas via mobile devices. As features contidas nesta base de dados são:

 * `step` - Passo: representa o total de horas transcorrido desde o início da simulação. Esta feature vai variar entre 1 e 95(4 dias);

 * `type`: Tipo: tipo de transação(depósito, saque, débito, pagamento e transferencia);

 * `amount`: Quantia: total que foi transacionado;

 * `nameOring`: ClienteOrigem: cliente que iniciou atransação;

 * `oldbalanceOrg`: SaldoInicialOrigem: saldo da conta de origem antes da transação;

 * `newbalanceOrig`: SaldoFinalOrigem: saldo da conta de origem depois da transação;

 * `nameDest`: ClienteDestino: cliente de destino da transação;

 * `oldbalandeDestino`: SaldoInicialDestino: saldo da conta destino antes da transação;

 * _`newbalanceDest`_: SaldoFinalDestino: saldo da conta destino depois da transação;

 * `isFraud`: Fraude: flag que define se a transação é fraudolenta ou não. Nesta simulação de fraude é a assumir se 'a conta do usuario foi   esvaziada e transferindo o dinherio para outra conta e depois sacado
 
 * `IssFlaggedFraud`: SinalizadaFraude: automaticamente categorizada como fraude pelo banco por fraude por tentar trasnferir mais de 200 mil em uma única conta


## 1.2 - RENAME COLUMNS

In [11]:
base_fraude_raw1.rename(columns={'step':'Passos', 'type':'Tipo', 'amount': 'Quantia',
                                'nameOrig':'ClienteOrigem', 'oldbalanceOrg': 'SaldoInicalOrig',
                                'newbalanceOrig':'SaldoFinalOrig', 'nameDest':'ClienteDest',
                                'oldbalanceDest':'SaldoInicialDest', 'newbalanceDest':'SaldoFinalDest',
                                'isFraud':'Fraude', 'isFlaggedFraud':'SinalizadaFraude'}, inplace=True)


## 1.3 - DATA DIMENSIONS

In [13]:
num_linhas, num_colunas = base_fraude_raw1.shape

print('Numero de linhas', num_linhas)
print('Numero de colunas', num_colunas)

Numero de linhas 557123
Numero de colunas 12


## 1.4 - DATA TYPES

In [12]:
#base_fraude_categoria = base_fraude_raw1[['Tipo','ClienteDest', 'ClienteOrigem']].astype('category')
base_fraude_raw1.dtypes

Unnamed: 0            int64
Passos                int64
Tipo                 object
Quantia             float64
ClienteOrigem        object
SaldoInicalOrig     float64
SaldoFinalOrig      float64
ClienteDest          object
SaldoInicialDest    float64
SaldoFinalDest      float64
Fraude                int64
SinalizadaFraude      int64
dtype: object

## 1.5 CHANGE TYPE

#### Não houve nenhuma mudança nos tipos de variaveis nessa etapa, pois todas as features se encontravam com seus tipos bem definidos

## 1.6 - CHECK N.A.

### Não foi encontrado nenhum valor nulo nesse Dataset

In [ ]:
base_fraude_raw1.isnull().sum()

Passos              0
Tipo                0
Quantia             0
ClienteOrigem       0
SaldoInicalOrig     0
SaldoFinalOrig      0
ClienteDest         0
SaldoInicialDest    0
SaldoFinalDest      0
Fraude              0
SinalizadaFraude    0
dtype: int64

In [ ]:
base_fraude_raw1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 557123 entries, 0 to 557122
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Passos            557123 non-null  int64  
 1   Tipo              557123 non-null  object 
 2   Quantia           557123 non-null  float64
 3   ClienteOrigem     557123 non-null  object 
 4   SaldoInicalOrig   557123 non-null  float64
 5   SaldoFinalOrig    557123 non-null  float64
 6   ClienteDest       557123 non-null  object 
 7   SaldoInicialDest  557123 non-null  float64
 8   SaldoFinalDest    557123 non-null  float64
 9   Fraude            557123 non-null  int64  
 10  SinalizadaFraude  557123 non-null  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 46.8+ MB


## 1.7 - FILL OUT N.A.

### Não havendo nenhum valor faltante nesse Dataset, não foi nescessario nenhum metodo de preenchimento ou exclusão dos mesmos

## 1.8 DISCRIPTIVE STATISTICAL

In [19]:
num_attributes = base_fraude_raw1.select_dtypes(include=['int64', 'float64'])
cat_attributes = base_fraude_raw1.select_dtypes(exclude=['int64', 'float64', 'datetime64[ns]'])



Medidas de tendencia central - 
 - Media
 - Mediana
 

Medidas de Dispersão

- Skewness
- Kurtosis
- Desvio Padrão
- Range (alcance)
- Min
- Max

### 1.8.1 - NUMERIC ESTATISTIC

In [21]:
m = show_descriptive_statistical(num_attributes)
m

,attributes,min,max,range,mean,median,std,skew,kurtosis
0,Unnamed: 0,0.000,557122.000,557122.000,278561.000,278561.000,160827.557,-0.000,-1.200
1,Passos,1.000,95.000,94.000,26.994,20.000,15.477,1.266,3.384
2,Quantia,0.100,10000000.000,9999999.900,163897.761,74913.810,275457.924,5.580,72.940
3,SaldoInicalOrig,0.000,19997409.980,19997409.980,539268.516,0.000,2551331.496,5.475,29.739
4,SaldoFinalOrig,0.000,20946317.020,20946317.020,551385.699,0.000,2579918.509,5.453,29.522
5,SaldoInicialDest,0.000,41482697.120,41482697.120,1102273.899,217585.310,2428607.923,5.068,37.892
6,SaldoFinalDest,0.000,42054659.730,42054659.730,1276203.964,321534.370,2577821.789,4.644,32.185
7,Fraude,0.000,1.000,1.000,0.001,0.000,0.025,39.803,1582.260
8,SinalizadaFraude,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


### 1.8.2 - CATEGORICAL ATRIBUTES



In [18]:
cat_attributes.apply(lambda x: x.unique().shape[0])

Tipo                  5
ClienteOrigem    557054
ClienteDest      269195
dtype: int64

## 2.0 - FEATURE ENGINER - PARTE I

In [22]:
#base_fraude_raw2 = base_fraude_raw1.copy()
#base_fraude_raw2.to_csv(HOME_PATH+INTERIM_DATA_PATH+'base_fraude_raw2.csv')
base_fraude_raw2 = pd.read_csv(HOME_PATH+INTERIM_DATA_PATH+'base_fraude_raw2.csv')

## 2.1 - MAPA MENTAL DE HIPOTESE

## 2.2 - CRIAÇÃO DAS HIPOTESE

## 2.3 - LISTA FINAL DE HIPOTESE

## 2.4 - FEATURE ENGINER - PARTE 2

## 3.0 - FILTRAGEM DE VARIAVEIS

## 3.1 - FILTRAGEM DE LINHAS

## 3.2 - FILTRAGEM DE COLUNAS - SELEÇÃO DAS COLUNAS